## Lab 08: Expenses and Budgeting

Welcome to the eigth lab of the course; we will do exercises on data structures and comprehension syntax simulating analysis of expenses.

## Exercise 1: Getting started

Use comprehension syntax on `list_expenses` to
- create a list where only the amount is kept
- create a list where only expenses made to Uber are kept
- create the set of dates in dd/mm/yyyy format (hint: split the date on '-' and concatenate the different pieces)
- create a list of tuples with two elements: the price (rounded to no decimals) and the length of the string describing the recipient (second element of the tuple); this, only if the tuple in `list_expenses` contains the word "util" in the category of the expense
- create the set of categories for which there is at least an expense of 60 euros

In [ ]:
list_expenses = [
    ('2024-10-01','Starbucks',5.75,'Food & Drink'),
    ('2024-10-05','Amazon',35.50,'Shopping'),
    ('2024-10-08','Local Restaurant',28.75,'Food & Drink'),
    ('2024-10-08','Uber',12.35,'Transportation'),
    ('2024-10-08','Electric Company',102.45,'Utilities'),
    ('2024-10-13','Walmart',68.20,'Groceries'),
    ('2024-10-13','Cinema',13.99,'Entertainment'),
    ('2024-10-13','Starbucks',4.50,'Food & Drink'),
    ('2024-10-22','Walmart',78.90,'Groceries'),
    ('2024-10-22','Water Utility',60.25,'Utilities'),
    ('2024-10-25','Amazon',25.99,'Shopping'),
    ('2024-10-25','Uber',10.75,'Transportation')
]

In [ ]:
[ t[2] for t in list_expenses ]
[ t for t in list_expenses if t[1]=='Uber']
{ t[0].split('-')[2]+"/"+t[0].split('-')[1]+t[0].split('-')[0] for t in list_expenses}
# alternative: { "/".join(reversed(item[0].split('-'))) for item in list_expenses }
# alternative: { "/".join(item[0].split('-')[::-1]) for item in list_expenses }
[ (round(t[2]), len(t[1])) for t in list_expenses if "util" in t[1].lower() ]
{ t[3] for t in list_expenses if t[2]>=60}

## Exercise 2: Merging expenses

If you have multiple bank accounts, each of themm will give you their own list of expenses, using different structures. The goal here is to merge them into a single, uniform list.

## 2.1 Mapping functions

Assume the existence of a mapping dictionary (see below) to be used to replace keys and values in the list of expenses. Each dictionary item is structured as 
    
    mapping['correct_key'] = (set_of_wrong_keys_to_be_replaced_with_correct_key, mapping_function_to_be_applied_to_values_with_wrong_keys)

Complete the mapping dictionary by:
- Creating a function that takes only the date part in a string that contains datetime field
    - E.g., `date_polish("2024-01-01 08:15")` must return '2024-01-01'
- Adding a lambda functiont to the 'amount' key that takes in input a number and returns its value rounded to 2 decimals

In [ ]:
def date_polish(date):
    return date[:10]

In [ ]:
date_polish("2024-01-01 08:15")

In [ ]:
mapping = dict()
mapping['date'] = ({'datetime'}, date_polish)
mapping['amount'] = ({'expense','value'}, lambda v: round(v,2))

## 2.2 Count the number of items that need to be resolved

- Create a set as the union of the values in the mapping dictionary
- Iterate on the lists to verify if an item contains one of the keys that should be replaced

In [ ]:
lists = [[
    {'date': '2024-10-12', 'recipient': 'Starbucks', 'description': 'Coffee', 'amount': 495.37, 'category': 'Food & Drink'},
    {'date': '2024-10-15', 'recipient': 'Starbucks', 'description': 'Coffee', 'amount': 484.96, 'category': 'Food & Drink'},
    {'date': '2024-10-05', 'recipient': 'Starbucks', 'description': 'Coffee', 'value': 338.41003, 'category': 'Food & Drink'}
],[
    {'datetime': '2024-10-17 09:00', 'recipient': 'Uber', 'description': 'Fuel', 'amount': 200.44545, 'category': 'Transportation'},
    {'datetime': '2024-10-10 10:30', 'recipient': 'Apple Store', 'description': 'Gadget purchase', 'amount': 69.46769, 'category': 'Electronics'},
    {'datetime': '2024-10-21 11:45', 'recipient': 'Amazon', 'description': 'Online purchase', 'expense': 213.89211, 'category': 'Shopping'}
]]

In [ ]:
combined_set = set()
for k in mapping.keys():
    combined_set = combined_set | mapping[k][0]
    
combined_set

In [ ]:
cnt = 0
for list in lists:
    for item in list:
        wrong_label_found = False
        for k in combined_set:
            if k in item.keys():
                wrong_label_found = True
                print(f'I found here that item {item} contains {k}')
        if wrong_label_found:
            # The counter is increased only once per item
            cnt += 1
cnt

In [ ]:
# Alternative, more compact version
cnt = 0
for list in lists:
    for item in list:
        if len(item.keys() & combined_set) > 0:
            cnt += 1
            print(f'I found here that item {item} contains something that labels {item.keys() & combined_set} must be changed')
cnt

## 2.3 Make consistent items

- Unify the lists: instead of a list of lists of items, make it a single list of items
- Iterate on the list to replace wrong keys with correct keys; use `item.pop(key)` to remove a key

In [ ]:
new_list = []
for l in lists:
    new_list.extend(l)
    # alternative: new_list += l

In [ ]:
# First solution: for every item, iterate on the dictionary to check whether the wrong keys exist in the item
for item in new_list:
    for correct_key in mapping.keys():
        for wrong_key in mapping[correct_key][0]:
            if wrong_key in item:
                item[correct_key] = mapping[correct_key][1](item[wrong_key])
                item.pop(wrong_key)

new_list

In [ ]:
# Second solution: for every item, iterate on its keys to check whether they are indicated in the mapping as wrong keys

# This solution may give errors on the third line
# In general, iterating over something (in this case, the mapping.keys()) that is being updated (in this case, by adding new keys) is not good practice
# Since sets do not have a fixed order, the error may or may not show up - you will see this by restarting the kernel multiple times
# for item in new_list:
#     for orig_key in item.keys():
#         for correct_key in mapping.keys():
#             if orig_key in mapping[correct_key][0]:
#                 item[correct_key] = mapping[correct_key][1](item[orig_key])
    
# To avoid the error above, new key-value pairs are first put in a temporary new_item and then added to the original item
for item in new_list:
    new_item = {}
    for orig_key in item.keys():
        for correct_key in mapping.keys():
            if orig_key in mapping[correct_key][0]:
                new_item[correct_key] = mapping[correct_key][1](item[orig_key])
    
    for new_key in new_item:
        item[new_key] = new_item[new_key]
        
new_list